In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Load the datasets
athlete_events_df = pd.read_csv('../Resources/DataFrames/Model_prep/Medals/athleteEvents_mf_model.csv')
gender_data_df = pd.read_csv('../Resources/DataFrames/Model_prep/Gender_wage_gap_model.csv')

In [3]:
gender_data_df

,Reference area,REF_AREA,TIME_PERIOD,Median_%_of_men_wages_gap
0,Australia,AUS,1980,18.750000
1,Australia,AUS,1979,20.000000
2,Australia,AUS,1978,19.791667
3,Brazil,BRA,2016,14.285714
4,Brazil,BRA,2015,15.384615
...,...,...,...,...
942,European Union (19 countries) in OECD,EU19OECD,2005,15.987233
943,European Union (19 countries) in OECD,EU19OECD,2004,16.940374
944,European Union (19 countries) in OECD,EU19OECD,2003,17.649478
945,Greece,GRC,2019,5.909091


In [4]:
athlete_events_df

,NOC,Team,Year,Sex,Medal,Count
0,AFG,Afghanistan,2008,M,Bronze,1
1,AFG,Afghanistan,2012,M,Bronze,1
2,ALG,Algeria,1992,F,Gold,1
3,ALG,Algeria,2000,F,Gold,1
4,ALG,Algeria,2008,F,Bronze,1
...,...,...,...,...,...,...
4505,ZIM,Zimbabwe,2004,F,Bronze,1
4506,ZIM,Zimbabwe,2004,F,Gold,1
4507,ZIM,Zimbabwe,2004,F,Silver,1
4508,ZIM,Zimbabwe,2008,F,Gold,1


In [5]:
merged_df = pd.merge(athlete_events_df, gender_data_df, left_on=['NOC', 'Year'], right_on=['REF_AREA', 'TIME_PERIOD'], how='inner')

In [6]:
merged_df

,NOC,Team,Year,Sex,Medal,Count,Reference area,REF_AREA,TIME_PERIOD,Median_%_of_men_wages_gap
0,AUS,Australia,1980,F,Bronze,1,Australia,AUS,1980,18.750000
1,AUS,Australia,1980,F,Gold,1,Australia,AUS,1980,18.750000
2,AUS,Australia,1980,M,Bronze,4,Australia,AUS,1980,18.750000
3,AUS,Australia,1980,M,Gold,5,Australia,AUS,1980,18.750000
4,AUS,Australia,1980,M,Silver,2,Australia,AUS,1980,18.750000
...,...,...,...,...,...,...,...,...,...,...
987,USA,United States-1,2016,F,Silver,1,United States,USA,2016,18.142077
988,USA,United States-1,2016,M,Silver,1,United States,USA,2016,18.142077
989,USA,United States-2,2016,F,Gold,1,United States,USA,2016,18.142077
990,USA,United States-2,2016,M,Bronze,2,United States,USA,2016,18.142077


In [7]:
medal_count_by_gender = merged_df.groupby(['NOC', 'Year', 'Sex']).agg({
    'Count': 'sum',  # Sum of medals won
    'Median_%_of_men_wages_gap': 'mean'  # Average wage gap for simplicity
}).reset_index()

In [8]:
medal_count_by_gender

,NOC,Year,Sex,Count,Median_%_of_men_wages_gap
0,AUS,1976,M,22,20.754717
1,AUS,1980,F,2,18.750000
2,AUS,1980,M,11,18.750000
3,AUS,1984,F,11,18.674699
4,AUS,1984,M,41,18.674699
...,...,...,...,...,...
380,USA,2012,M,88,19.086651
381,USA,2014,F,39,17.451206
382,USA,2014,M,25,17.451206
383,USA,2016,F,149,18.142077


In [10]:
men_df = medal_count_by_gender[medal_count_by_gender['Sex'] == 'M']
women_df = medal_count_by_gender[medal_count_by_gender['Sex'] == 'F']

In [11]:
correlation_men = men_df[['Count', 'Median_%_of_men_wages_gap']].corr().iloc[0, 1]
correlation_women = women_df[['Count', 'Median_%_of_men_wages_gap']].corr().iloc[0, 1]

In [12]:
print(f"Correlation between gender wage gap and medal count (Men): {correlation_men}")
print(f"Correlation between gender wage gap and medal count (Women): {correlation_women}")

Correlation between gender wage gap and medal count (Men): 0.19484673476181982
Correlation between gender wage gap and medal count (Women): 0.08388847649617107


In [13]:
features = medal_count_by_gender.drop(columns=['Count', 'NOC', 'Sex', 'Year'])
target = medal_count_by_gender['Count']

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [17]:
# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [18]:
# Train the model
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [19]:
# Make predictions
y_pred = rf.predict(X_test)

In [20]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 20.674439393939394


In [25]:
# Function to calculate correlation for each country and gender
def calculate_country_correlation(group):
    return group[['Count', 'Median_%_of_men_wages_gap']].corr().iloc[0, 1]

In [26]:
# Calculate correlation for each country and gender
country_gender_correlation = medal_count_by_gender.groupby(['NOC', 'Sex']).apply(calculate_country_correlation).reset_index()

In [27]:
# Rename the correlation column for clarity
country_gender_correlation = country_gender_correlation.rename(columns={0: 'Correlation'})

In [29]:
# Display the correlation for each country and gender
country_gender_correlation

,NOC,Sex,Correlation
0,AUS,F,-0.300004
1,AUS,M,-0.214345
2,AUT,F,-0.290925
3,AUT,M,0.121380
4,BEL,F,0.419562
5,BEL,M,-0.695070
6,BRA,F,1.000000
7,BRA,M,-1.000000
8,CAN,F,-0.452400
9,CAN,M,-0.326202
